# **Test 080424**

In [1]:
import pandas as pd
import json
import csv
import os

In [2]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:\
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = json.loads(str_text)
                print(raw_sample)
                data.append(raw_sample)
            except:
                print(line)
                pass
    return data

def read_and_split_data(file_path, batch_size=10000):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    small_list_data = []
    count = 0
    with open(file_path, "r") as f:
        for line in f.readlines():
            if count < batch_size:
                # 过长的评论文本会超出io限制报错。 暂时忽略
                try:
                    # 字符串预处理
                    # 布尔替换成python的习惯
                    str_text = line.replace("true", "True")
                    str_text = str_text.replace("false", "False")
                    # 转成字典形式
                    raw_sample = eval(str_text)
                    data.append(raw_sample)
                    count += 1
                except:
                    pass
            else:
                small_list_data.append(data)
                data = []
                count = 0
    small_list_data.append(data)
    return small_list_data

def read_txt_file_to_list(file_path):
    records = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            records.append(line.strip())
    return records


def remove_duplicate_rows(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'asin', 'reviewerID'])
    return df

def remove_duplicate_rows_ver2(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'itemID', 'reviewerID'])
    return df

In [3]:
json_file_path = "./DeepCGSR_triet/data/raw/Digital_Music_5.json"  # Thay đổi đường dẫn tới file JSON của bạn
output_directory = "./DeepCGSR_triet/data/DigitalMusic"  # Thay đổi đường dẫn thư mục lưu file CSV của bạn
chunk_size = 10000  # Kích thước của mỗi phần dữ liệu

small_list_data = read_and_split_data(json_file_path, chunk_size)
count_name = 0
for data in small_list_data:
    # Chuyển đổi dữ liệu thành DataFrame
    data_df = pd.DataFrame(data)
    print(data_df)
    # data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

    # Lưu dữ liệu dưới dạng CSV
    output_file_path = os.path.join(output_directory, f"Small_Digital_Music_{len(data_df)}_{count_name}.json")
    data_df.to_json(output_file_path, orient='records', lines=True)

    print(f"Saved {len(data_df)} records to {output_file_path}")
    count_name += 1

      overall vote  verified   reviewTime      reviewerID        asin  \
0         5.0    3      True   06 3, 2013  A2TYZ821XXK2YZ  3426958910   
1         5.0  NaN      True  10 11, 2014  A3OFSREZADFUDY  3426958910   
2         5.0  NaN      True  02 11, 2014  A2VAMODP8M77NG  3426958910   
3         4.0    3     False   12 7, 2013   AAKSLZ9IDTEH0  3426958910   
4         5.0  NaN      True  06 12, 2016  A3OH43OZJLKI09  5557706259   
...       ...  ...       ...          ...             ...         ...   
9995      5.0  NaN      True   01 4, 2015  A1KBHQWJYKYA01  B000WOXPNS   
9996      5.0  NaN      True  11 28, 2014  A1KRWR5D81AXJE  B000WOXPNS   
9997      5.0  NaN      True   11 4, 2014  A2G6VF7XDPKFG3  B000WOXPNS   
9998      5.0  NaN      True  07 30, 2014  A3QXAIQ3EHY7AD  B000WOXPNS   
9999      5.0  NaN      True  06 16, 2014  A13ZE9XWRQPPYC  B000WOXPNS   

                          style       reviewerName  \
0      {'Format:': ' Audio CD'}            Garrett   
1      {'Format

In [4]:
data = pd.read_json("DeepCGSR_triet/data/DigitalMusic/Small_Digital_Music_10000_3.json", lines=True)
print(data)
data_df = pd.DataFrame(data, columns=['reviewerID', 'asin', 'overall', 'reviewText'])
print(data_df.head())
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

data_rating = data_df["overall"].tolist()
data_review = data_df["reviewText"].tolist()
data_reviewerID = data_df["reviewerID"].tolist()
data_itemID = data_df["asin"].tolist()

      overall  verified   reviewTime      reviewerID        asin  \
0           5      True  01 13, 2013  A1E78KIR64YBT6  B000VZO4TW   
1           5      True  03 15, 2016   ASUDG3BDP87CI  B000VZO4TW   
2           5      True  02 21, 2016   AKMWILI3V2XCY  B000VZO4TW   
3           5      True  02 10, 2016   AAAVOYJKW8IC8  B000VZO4TW   
4           5      True   02 7, 2016  A2HPG8QACV8QAJ  B000VZO4TW   
...       ...       ...          ...             ...         ...   
9995        5     False  12 25, 2004   A11GY1G10WSE8  B00123KEGO   
9996        5     False  11 18, 2004   AMG2LF8MLUDN1  B00123KEGO   
9997        5      True  10 11, 2014  A134WU6PV3QFSW  B00123LZT4   
9998        3      True  09 28, 2014  A2Y2CLBUBP0QNE  B00123LZT4   
9999        5      True  08 21, 2014  A15A0G4ZK7RGY4  B00123LZT4   

                          style   reviewerName  \
0     {'Format:': ' MP3 Music'}    lil_sparrow   
1     {'Format:': ' MP3 Music'}    john basile   
2     {'Format:': ' MP3 Music'}  

In [5]:
print(len(data_rating))
print(len(data_reviewerID))

10000
10000


In [6]:
import pandas as pd
# step01:
def defineRating(data_rating):
  T_v = []
  for rating in data_rating:
    if rating >= 4:
      T_v.append(1)
    else:
      T_v.append(-1)
  return T_v

T_v = defineRating(data_rating)

# Define the data for the dataframe
data = {'id': range(len(T_v)), 'T_v': T_v}

# Create the dataframe
df_rating = pd.DataFrame(data)

# Print the dataframe
print(df_rating)


        id  T_v
0        0    1
1        1    1
2        2    1
3        3    1
4        4    1
...    ...  ...
9995  9995    1
9996  9996    1
9997  9997    1
9998  9998   -1
9999  9999    1

[10000 rows x 2 columns]


In [7]:
# Thêm vào dataframe df_rating cột C_v
df_rating['reviewerID'] = data_reviewerID
df_rating['asin'] = data_itemID
df_rating['overall'] = data_rating
df_rating['reviewText'] = data_review
# Loại bỏ các dòng có reviewText trùng lặp
# df_rating = remove_duplicate_rows(df_rating)
print(df_rating)

        id  T_v      reviewerID        asin  overall  \
0        0    1  A1E78KIR64YBT6  B000VZO4TW        5   
1        1    1   ASUDG3BDP87CI  B000VZO4TW        5   
2        2    1   AKMWILI3V2XCY  B000VZO4TW        5   
3        3    1   AAAVOYJKW8IC8  B000VZO4TW        5   
4        4    1  A2HPG8QACV8QAJ  B000VZO4TW        5   
...    ...  ...             ...         ...      ...   
9995  9995    1   A11GY1G10WSE8  B00123KEGO        5   
9996  9996    1   AMG2LF8MLUDN1  B00123KEGO        5   
9997  9997    1  A134WU6PV3QFSW  B00123LZT4        5   
9998  9998   -1  A2Y2CLBUBP0QNE  B00123LZT4        3   
9999  9999    1  A15A0G4ZK7RGY4  B00123LZT4        5   

                                             reviewText  
0     my first favorite kiss song off the first favo...  
1                                             Excellent  
2     I don't like a bunch of Kiss, but I like this ...  
3     Loved the song.  Not a Kiss fan, but this song...  
4                                    

# Pre-Process Dataset


In [8]:
import nltk
import math
import spacy
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import TextClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

pos_words = read_txt_file_to_list("./DeepCGSR_triet/data/positive-words.txt")
neg_words = read_txt_file_to_list("./DeepCGSR_triet/data/negative-words.txt")
print(pos_words)
print(neg_words)
a = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
brands = ['dove', 'aveda', 'silver']
keywords = ['overall', 'summary']

tagger = Classifier.load('pos')
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")
def remove_stopwords(sentence):
    """
    Loại bỏ các từ dừng từ câu sử dụng NLTK.

    Parameters:
    - sentence (str): Câu cần loại bỏ từ dừng.

    Returns:
    - str: Câu sau khi loại bỏ các từ dừng.
    """
    # Tách câu thành các từ
    words = word_tokenize(sentence)

    # Tải danh sách từ dừng từ NLTK
    stop_words = set(stopwords.words('english'))

    # Loại bỏ các từ dừng
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Ghép lại các từ thành câu mới
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# df_rating['reviewText'] = df_rating['reviewText'].apply(remove_stopwords)

def get_sentiment_words(text):
    # Khởi tạo SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách câu thành các từ
    words = nltk.word_tokenize(text)

    # Lấy danh sách các từ ngừng (stop words)
    stop_words = set(stopwords.words('english'))

    # Danh sách các từ có cảm xúc
    sentiment_words = []

    # Kiểm tra từng từ trong câu
    for word in words:
        print(word, " - ", sia.polarity_scores(word)['compound'])
        # Nếu từ không phải là từ ngừng và có điểm tính cảm từ VADER lớn hơn 0
        # if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] > 0:
        if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] != 0:
            sentiment_words.append(word)

    return sentiment_words


def sentiment_analysis_flair(sentence):
    # Kiểm tra xem câu có rỗng không
    if not sentence.strip():
        return 0.0
    # Load pretrained sentiment model
    classifier = TextClassifier.load('en-sentiment')

    # Create a Flair Sentence
    flair_sentence = Sentence(sentence)

    # Predict sentiment
    classifier.predict(flair_sentence)
    # print(flair_sentence)
    # Get sentiment value (POSITIVE/NEGATIVE) and score
    sentiment_value = flair_sentence.labels[0].value
    sentiment_score = flair_sentence.labels[0].score
    score = 0.0
    bias = 1.0
    for keyword in keywords:
        if keyword.lower() in sentence.lower():
            bias = 1.75
    
    if(sentiment_value == 'POSITIVE'):
        score = sentiment_score * bias
    else:
        score = -sentiment_score * bias

    return score

#Define the text to analyze

def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    # print(text)
    if not text:
        return -1
    text = text[:512]
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()
    # print(probs)
    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

def check_string(input_value):
    # Kiểm tra xem giá trị có phải là chuỗi hay không
    if isinstance(input_value, str):
        # Nếu là chuỗi, trả về giá trị không thay đổi
        return input_value
    else:
        # Nếu không phải là chuỗi, trả về chuỗi rỗng
        return ''

def seperate_sentences(text):
    text = check_string(text)
    doc = nlp(text)
    # Khởi tạo một danh sách để lưu trữ các câu
    sentences = []
    current_sentence = ""
    # Duyệt qua mỗi từ trong văn bản
    for token in doc:
        # Nếu từ hiện tại không phải là dấu kết thúc câu, thêm từ vào câu hiện tại
        
        if token.is_sent_start:
            # Nếu có câu hiện tại, thêm câu vào danh sách câu
            if current_sentence:
                sentences.append(current_sentence.strip())
            # Bắt đầu một câu mới
            current_sentence = token.text
        elif token.text in ["but", "and"]:
            # print("that but:", current_sentence)
            if (brand.lower() in current_sentence.lower() for brand in brands):
                # print("need continue: ", current_sentence)
                current_sentence += " " + token.text
                continue
            elif current_sentence:
                sentences.append(current_sentence.strip())
                current_sentence = token.text
        else:
            # Thêm từ vào câu hiện tại
            current_sentence += " " + token.text
    # Thêm câu cuối cùng vào danh sách câu
    if current_sentence:
        sentences.append(current_sentence.strip())
    return sentences

def split_sentences_and_filter_sentiment(text):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách đoạn văn thành các câu nhỏ
    # sentences = nltk.sent_tokenize(text)
    sentences = seperate_sentences(text)
    if len(sentences) == 1 and not re.search(r'[.!?]', text):
        sentences = [text]
    # print(sentences)
    # Lọc các câu không chứa bất kỳ từ miêu tả cảm xúc nào
    filtered_sentences = []
    for sentence in sentences:
        # print("---- sentence ------")
        if not sentence.strip():
            continue
        sentence = sentence.strip()
        words = nltk.word_tokenize(sentence)
        
        # for word in words:
        #     if(sia.polarity_scores(word)['compound'] != 0 ):
        #         print(word, ":", sia.polarity_scores(word)['compound'])

        contains_sentiment = (any(sia.polarity_scores(word)['compound'] != 0 for word in words) or 
                              any(word.lower() in pos_words or word.lower() in neg_words for word in words))
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
        # Flair
        
        sentence_flair = Sentence(sentence)
        tagger.predict(sentence_flair)
        # print(sentence_flair.tokens)
        is_a = False
        for token in sentence_flair.tokens:
            for label in token.get_labels('pos'):
                if label.value in a:
                    # print(token, label.value)
                    is_a = True
                    break
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
                is_a = False
        #--------------
        if contains_sentiment:
            filtered_sentences.append(sentence)
        elif is_a:
            filtered_sentences.append(sentence)
        else:
            print("remove: ", sentence)
            # print(evaluate_sentiment_flair(sentence))
        # print("---- end sentence ------")
    # Thêm câu vào danh sách nếu không có câu nào chứa từ miêu tả cảm xúc
    if not filtered_sentences:
        filtered_sentences = [""]
    # print(filtered_sentences)
    return filtered_sentences

/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-28 18:26:15.546965: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them 

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [80]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

predictRating = []
text = "Luv this song!!"
# text = remove_stopwords(text)


sentences = split_sentences_and_filter_sentiment(text)
total_sentiment = 0
for sentence in sentences:
    # sentence = remove_stopwords(sentence)
    score = sentiment_analysis_flair(sentence)
    print(sentence, score)
    total_sentiment += score
    # print(total_sentiment)
print(total_sentiment)
if total_sentiment > 0.0:
    predictRating.append(1)
    print("Result: 1")
else:
    predictRating.append(-1)
    print("Result: -1")

[Token[0]: "Luv" → NNP (0.9986), Token[1]: "this" → DT (1.0), Token[2]: "song" → NN (0.9983), Token[3]: "!" → . (0.9984), Token[4]: "!" → . (0.9767)]
Sentence[5]: "Luv this song ! !" → POSITIVE (0.9994)
Luv this song ! ! 0.9994039535522461
0.9994039535522461
Result: 1


In [10]:
import tqdm
import os


predictRating = []
filtered_sentences = []
def process_and_save(df_rating, output_file):

    output_df = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReview'])
    score = 0
    # Đoạn mã dùng tqdm để hiển thị tiến trình %
    for i, row in tqdm.tqdm(df_rating.iterrows(), desc="Processing", total=len(df_rating)):
        sentences = split_sentences_and_filter_sentiment(row['reviewText'])
        #-------1------
        if(len(sentences) > 1):
            filtered_sentence = ". ".join(sentences)
        else:
            filtered_sentence = row['reviewText']
        filtered_sentences.append(filtered_sentence)
        #-------2-------
        total_sentiment = sum(sentiment_analysis_flair(sentence) for sentence in sentences)
        if total_sentiment > 0.0:
            score = 1
        else:
            score = -1
        predictRating.append(score)
        #-------3-------
        # Lưu vào DataFrame và ghi vào file CSV sau mỗi 10 record
        output_df.loc[len(output_df)] = { 'reviewerID': row['reviewerID'],
                                          'asin': row['asin'],
                                          'predictRating': score,
                                          'T_v': row['T_v'],
                                          'overall': row['overall'],
                                          'reviewText': row['reviewText'],
                                          'filteredReviewText': filtered_sentence }
        
        if (i + 1) % 100 == 0:
            output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            output_df = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReviewText'])
    
    # Ghi phần còn lại của DataFrame vào file CSV
    if not output_df.empty:
        output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)


# Sử dụng hàm
process_and_save(df_rating, './DeepCGSR_triet/output_DigitalMusic_03.csv')


Processing:   0%|          | 8/10000 [00:26<10:05:26,  3.64s/it]

remove:  In the 1970 - 80s era , they embodied an aspect of rock alongside the Who , the Stones and the others .
remove:  It begins : 

 Beth , I hear you callin '


Processing:   0%|          | 13/10000 [01:28<17:23:51,  6.27s/it]

remove:  This was the way my hubbie and i were .


Processing:   0%|          | 14/10000 [01:33<16:09:32,  5.83s/it]

...

Processing:  32%|███▏      | 3228/10000 [4:45:45<5:11:37,  2.76s/it]

remove:  :)
remove:  Give it a try !


Processing:  32%|███▏      | 3241/10000 [4:46:20<4:31:52,  2.41s/it]

remove:  Quality product , quality service .


Processing:  32%|███▏      | 3245/10000 [4:46:26<3:25:51,  1.83s/it]

In [12]:
output = pd.read_csv('./DeepCGSR_triet/output.csv')
output_df = pd.DataFrame(output)
print("Length Original Dataset: ", len(output_df))
outliner = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText'])
filteredRecords = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText'])
for i, row in output_df.iterrows():
    if row['predictRating'] != row['T_v']:
        outliner.loc[len(outliner)] = row
    else:
        filteredRecords.loc[len(filteredRecords)] = row
        # print('------------------------')
outliner.to_csv('./DeepCGSR_triet/outliner_DM.csv', index=False)
filteredRecords.to_csv('./DeepCGSR_triet/filteredRecords_DM.csv', index=False)

print("Length Outliner Dataset: ", len(outliner))
print("Length Filered Dataset: ", len(filteredRecords))

Length Original Dataset:  10010
Length Outliner Dataset:  944
Length Filered Dataset:  9066


In [19]:
import pandas as pd
import json
from tqdm import tqdm


#Load json data to dataframe
raw_data = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_5.json', lines=True)
print("Dataset: ", len(raw_data))

count = 0
result_data = raw_data.copy(deep=False)
result_data = result_data.iloc[0:0]

for filter_index, filter_row in tqdm(filteredRecords.iterrows(), total=len(filteredRecords), desc="Processing"):
    for index, row in raw_data.iterrows():
        if (row['reviewerID'] == filter_row['reviewerID'] and row['asin'] == filter_row['asin'] 
            and row['reviewText'] == filter_row['reviewText']):
            result_data.loc[len(result_data)] = row
            count += 1
            # print(count, row['asin'], row['reviewerID'], row['reviewText'])
            break

print("Filtered dataset: ", len(result_data))
result_data.to_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_Filtered.json', orient='records', lines=True)

data_filtered = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_Filtered.json', lines=True)



Dataset:  169781


Processing: 100%|██████████| 9066/9066 [28:25<00:00,  5.32it/s]  


Filtered dataset:  8957
